In [2]:
%pip install ibapi


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ib-insync


     |████████████████████████████████| 72 kB 131 kB/s eta 0:00:01
  Using cached backports.zoneinfo-0.2.1-cp38-cp38-manylinux1_x86_64.whl (74 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol
ticker_symbol = 'AAPL'

# Define the start and end dates
start_date = '1990-01-01'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Fetch the data from Yahoo Finance
apple_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the first few rows of the data
print(apple_data.head())

# Save the data to a CSV file
apple_data.to_csv('apple_data_1990_to_date.csv')


[*********************100%%**********************]  1 of 1 completed


                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
1990-01-02  0.314732  0.334821  0.312500  0.332589   0.262418  183198400
1990-01-03  0.339286  0.339286  0.334821  0.334821   0.264179  207995200
1990-01-04  0.341518  0.345982  0.332589  0.335938   0.265060  221513600
1990-01-05  0.337054  0.341518  0.330357  0.337054   0.265941  123312000
1990-01-08  0.334821  0.339286  0.330357  0.339286   0.267702  101572800
